<center><img src="media/imagenes/Banner.png" height = 100 width = 1000></center>

Este notebooj pretende enseñar el resultadod de todas las implementaciones sin meterse mucho en los tedalles de la implementación, pretende ser una guia visual e interactiva, para endender todas las decisiones de diseño que hemos tomado en el proyecto. Esta pensado para ser ejecutado celda a celda salvo que se diga lo contrario.

In [1]:
import dependencias
from pprint import pprint

from dependencias import enfrentar

pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Reglas

Hay que enseñar lo que hemos implentado del juego base

## Agentes

<img src="media/imagenes/euristicos.png" style="float: right;">

### Agentes Basados en Heuristicas.

Los agentes que vamos a enseñar en este apartado son agentes que se basan en una politica de recompensa para decidir que movimiento realizar en cada turno, estan implementados mediante una clase abstracta que tiene los siguientes metodos:

- get_reward: Calcula la recompensa asociada a cada movimiento en funcion de una politica de recompensa.
- get_action: Calcula la mejor acción a realizar en función de la recompensa obtenida.

de este modo podemos empezar a jugar con el agente y ver como se comporta.



A continuacion vamos a enseñar los agentes que hemos implementado y como se comportan en el juego.

## Agentes Chaser

La politica de recompensa de este agente se basa en la distancia entre la cabeza de la serpiente y la comida mas cercana, de este modo el agente siempre intentará acercarse a la comida, aunque esto implique que la serpiente se muera.

<center><img src="media/imagenes/chasser.png" style="float: center;"></center>

In [2]:
agente_chaser = dependencias.Agentes.ChaserAgent()

In [3]:
# Ejecuta esta celda para ver como se comporta el agente puedes modificar los parametros del juego para ver como se comporta en distintos escenarios

SIZE = (15, 15)
N_FOODS = 5

game = dependencias.Snake_game(SIZE, N_FOODS, agente_chaser)
game.play_with_pygame()

In [4]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 1511.64it/s]

{'movimientos por puntuacion': 4,
 'movimientos_maximos': np.int64(123),
 'movimientos_medios': np.float64(32.28),
 'movimientos_minimos': np.int64(7),
 'proporcion_del_tablero_ocupada': 0.04,
 'puntuacion_maxima': np.int64(26),
 'puntuacion_media': np.float64(7.23),
 'puntuacion_minima': np.int64(3)}


### Conclusiones

Este agente logra su objetivo de llegar muy rapido a por la comida (movimientos por puntuacion), pero no consigue sobrevivir mucho tiempo.

### Mejoras

Este agente podría ser mejorado si se le añade una penalización por muerte inmediata.

## Agentes Avoider

La politica de recompensa de este agente se basa en evitar las muertes inediatas, ya sea contra la serpriente o contra las paredes, asociando una penalización a estos movimientos.

<center><img src="media/imagenes/inminent.png" style="float: center;"></center>

In [5]:
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()

In [6]:
# Ejecuta esta celda para ver como se comporta el agente
SIZE = (15, 15)
N_FOODS = 5

game = dependencias.Snake_game(SIZE, N_FOODS, agente_avoider)
game.play_with_pygame()

In [7]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 385.62it/s]

{'movimientos por puntuacion': 61,
 'movimientos_maximos': np.int64(1191),
 'movimientos_medios': np.float64(600.67),
 'movimientos_minimos': np.int64(195),
 'proporcion_del_tablero_ocupada': 0.03,
 'puntuacion_maxima': np.int64(18),
 'puntuacion_media': np.float64(9.81),
 'puntuacion_minima': np.int64(5)}


### Conclusiones

Este agente majora mucho su tiempo de vida, pero no consigue alcanzar la comida de manera eficiente.

### Mejoras

Ademas vemos que con evitar muertes inmediatas no es suficiente, puesto que hay escenarios en los que te puedes quedar encerrado.



## Combinacion de agentes

Esta implementation de los agentes tan particular, nos permite combinar las recompensas que le dan a cada movimiento distintos agentes para conseguir un agente que se comporte de una manera más eficiente.

<img src="media/imagenes/combinado.png" style="display: block; margin: auto;" width="400">



In [8]:
chouder = dependencias.Agentes.Combined_agent((agente_avoider, agente_chaser), (1, 0.5))

In [9]:
# Ejecuta esta celda para ver como se comporta el agente
game = dependencias.Snake_game((15, 15), 5, chouder)
game.play_with_pygame()

In [10]:
estadisticas = game.evaluar()
pprint(estadisticas)

100%|██████████| 100/100 [00:00<00:00, 189.74it/s]

{'movimientos por puntuacion': 5,
 'movimientos_maximos': np.int64(378),
 'movimientos_medios': np.float64(193.22),
 'movimientos_minimos': np.int64(65),
 'proporcion_del_tablero_ocupada': 0.12,
 'puntuacion_maxima': np.int64(68),
 'puntuacion_media': np.float64(35.98),
 'puntuacion_minima': np.int64(12)}


### Conclusiones

El comportaiento de este agente es mucho mejor que el de los agentes individuales, y parece una buena estrategia para el inicio del juego.

### Mejoras

El agente sigue sin ser capaz de evitar quedarse encerrado, hay que añadir una heuristica mas a futuro para evitar este tipo de situaciones.

<img src="media/imagenes/encerrado.png" style="display: block; margin: auto;" width="400">

*en el ejemplo moverse a la derecha provoca una muerte inevitable pero el agente no lo considera una mala jugada*

## Politicas basadas en busqueda en anchura (Estrategias a largo plazo)

para hacer decisiones a largo plazo se nos plantean dos opciones.

- Fuerza bruta (Exploracion en estados): podemos ver a donde nos lleva una combinacion de movimientos y ver cual es mejor.

problema: el espacio de estados es muy grande y no podemos explorar todas las posibilidades, tendriamos que seleccionar una profundidad maxima y explorar hasta ahi. Además relentizaria cada simulacion.

- Busqueda en anchura: Lanzando busqueda en anchura desde los movimientos posibles en cada turno podemos ver cual es el mejor movimiento a largo plazo, evaluando algunas caracteristicas del tablero.

A continuacion desarrollamos agentes basados en busqueda en anchura, para solucionar probleas de muerte a largo plazo.

## Agentes Searcher

Este agente lanza 3 busquedas en anchura, empezando en cada una las casillas accedibles en el siguiente movimiento, aprovecha para contar el numer de casillas accesibles en cada caso y asigna una puntuacion a cada movimiento en funcion de este valor.

<div style="text-align: center;">
$$
W = \frac{\text{casillas accesibles en el movimiento } m}{\text{total casillas accesibles}}
$$
</div>

*De este modo premiamos los movimientos que te llevan a los espacios abiertos*


In [11]:
agente_searcher = dependencias.Agentes.Busqueda_anchura()
game = dependencias.Snake_game((15, 15), 5, agente_searcher)

In [12]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame()

A continuacion definimos una mezcla de los 3 agentes anteriores, para ver como se comporta en el juego.

In [13]:
agente_searcher = dependencias.Agentes.Busqueda_anchura()
agente_avoider = dependencias.Agentes.Avoid_inmediate_death()
agente_chaser = dependencias.Agentes.ChaserAgent()

deep_chouder = dependencias.Agentes.Combined_agent((agente_avoider, agente_chaser, agente_searcher), (1, 0.2, 3))
game = dependencias.Snake_game((15, 15), 5, deep_chouder)

In [14]:
# Ejecuta esta celda para ver como se comporta el agente

game.play_with_pygame()

#### Comportamiento del agente

- El comportamiento de este agente es muy bueno, consigue sobrevivir mucho tiempo y llegar a la comida de manera eficiente, además cuando se queda encerrado sigue opcupando espacio hasta generar una salida y una vez existe es capaz de escapar generando muchas huidas interesantes.

In [ ]:
# lo comparamos con el agente que desarrollamos antes.
# Esta celda puede tardar un poco en ejecutarse se recomienda siplemente mirar los resultados.

dependencias.enfrentar(deep_chouder, chouder, n_partidas=100)

  0%|          | 0/100 [00:00<?, ?it/s]

Estas estaditicas muestran que deep_chouder es un agente mucho mejor que chouder, consigue sobrevivir mucho mas tiempo y llegar a mas comida. Sin embargo logicamente chouder es as eficiente y crece mas rapido.

## Politicas basadas en busqueda en anchura (Estrategias a largo plazo)

In [1]:
import dependencias

pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
tail_chasser = dependencias.Agentes.Tail_Chasser()
avoider = dependencias.Agentes.Avoid_inmediate_death()
chasser = dependencias.Agentes.ChaserAgent()
searcher = dependencias.Agentes.Busqueda_anchura()
filleter = dependencias.Agentes.Filler()
cycle_detector = dependencias.Agentes.Cycle_detector(avoid_skipable_loops=False)

In [3]:
combinado = dependencias.Agentes.Combined_agent(agentes = (tail_chasser, chasser, avoider, filleter, searcher, cycle_detector), weights= (10, 0.5, 0.55, 0.05, 0, 100))
game = dependencias.Snake_game((15, 15), 5, combinado)

In [8]:
game.play_with_pygame()

In [4]:
game.evaluar(100)

  0%|          | 0/100 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [4]:
dependencias.enfrentar(combinado, chasser, n_partidas=2)

100%|██████████| 2/2 [00:00<00:00, 2003.49it/s]

         Estadisticas          |    Agente 1     |    Agente 2    
------------------------------ | --------------- | ---------------
puntuacion_media               |   ((*221.0*))   |       6.0      
puntuacion_maxima              |    ((*222*))    |        7       
puntuacion_minima              |    ((*220*))    |        5       
movimientos_medios             |  ((*3328.0*))   |      25.0      
movimientos_maximos            |   ((*3448*))    |       32       
movimientos_minimos            |   ((*3208*))    |       18       
movimientos por puntuacion     |    ((*15*))     |        4       
------------------------------ | --------------- | ---------------


In [12]:
game.evaluar(100)

100%|██████████| 100/100 [03:36<00:00,  2.17s/it]


{'puntuacion_media': np.float64(208.62),
 'puntuacion_maxima': np.int64(222),
 'puntuacion_minima': np.int64(16),
 'movimientos_medios': np.float64(3515.48),
 'movimientos_maximos': np.int64(6807),
 'movimientos_minimos': np.int64(110),
 'movimientos por puntuacion': np.float64(16.851116863196243)}